In [1]:
import os
import errno
import torch
import warnings
warnings.filterwarnings(action = 'ignore')
import transformers

from datasets import load_dataset
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

/home/ubuntu/qlora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/qlora/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so.11.0
CUDA SETUP: Loading binary /home/ubuntu/qlora/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


In [2]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [3]:
data = load_dataset('beomi/KoAlpaca-v1.1a')

Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 110.62it/s]


In [4]:
data['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [5]:
data = data.map(
    lambda x: {"text" : f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-0a252a9f46a47ed5.arrow


In [6]:
data['train']

Dataset({
    features: ['instruction', 'output', 'url', 'text'],
    num_rows: 21155
})

In [7]:
model_path = './model_file/polyglot-ko-12.8b'

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,
                                            quantization_config = bnb_config,
                                            device_map = {"":0})

In [8]:
!nvidia-smi

Wed Jul 12 11:38:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   33C    P0    58W / 250W |   7955MiB / 40960MiB |      5%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:00:07.0 Off |                  Off |
| N/A   

In [9]:
data = data.map(lambda x: tokenizer(x['text']), batched = True)
# data = data.remove_columns(['instruction', 'output', 'url'])

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-c6197d89611d28b3.arrow


In [10]:
data['train']

Dataset({
    features: ['instruction', 'output', 'url', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 21155
})

- kbit training에 적재

In [11]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [12]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f'trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}')

In [13]:
config = LoraConfig(
    r = 8,
    lora_alpha = 32,
    target_modules = ['query_key_value'],
    lora_dropout = .05,
    bias = "none",
    task_type = 'CAUSAL_LM'
)

model = get_peft_model(model, config)
print_trainable_parameters(model)
model.print_trainable_parameters()

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424
trainable params: 6,553,600 || all params: 6,608,701,440 || trainable%: 0.09916622894073424


In [14]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=15360, bias=False)
                

In [15]:
!nvidia-smi

Wed Jul 12 11:38:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   32C    P0    37W / 250W |   9165MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:00:07.0 Off |                  Off |
| N/A   

In [17]:
# gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

# making model outpath 
model_outpath = './model_file/qlora_PoC_12.8b'
mkdir_p(model_outpath)

args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    #per_device_eval_batch_size = 16,  ## val_dataset 없음
    #warmup_steps = 500,
    max_steps = 1000,
    learning_rate = 2e-8,
    fp16 = True,
    logging_steps = 10,
    output_dir = model_outpath,
    optim = 'paged_adamw_8bit',
    report_to = 'wandb',
    run_name = 'qlora_PoC_12.8b'
)

trainer = Trainer(
    model = model,
    train_dataset = data['train'],
    args = args,
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
)

In [ ]:
model.config.use_cache = False  # silence the warnings. plz re-enable for inference!
trainer.train()

wandb: Currently logged in as: dongryeol. Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.432700
20,2.408700
30,2.417800
40,2.420000
50,2.430900
60,2.411300
70,2.418900
80,2.440900
90,2.418000
100,2.441300
